In [1]:
import cv2 
import numpy as np

In [2]:
# Identify corners through Psi-tomasi corner detector
def find_features(gray_image):
    points = cv2.goodFeaturesToTrack(gray_image, mask=None, maxCorners = 100, qualityLevel = 0.4, 
                                     minDistance = 10, blockSize = 7)
    return points

In [3]:
# Lucas Kannade Sparse Optical flow model
def LK_model(gray_t_1, gray_t, prev_points):
    points, status, error = cv2.calcOpticalFlowPyrLK(gray_t_1, gray_t, prev_points, None, winSize  = (10, 10), maxLevel = 2,
                                                     criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    return (points, status, error)

In [4]:
# Driver script

filename = "Traffic_video_input.mp4"

# Assign a random color to show the optical flow.
color = np.random.randint(0, 255, (100, 3))
capture = cv2.VideoCapture(filename)


frame_width = int(capture.get(3))
frame_height = int(capture.get(4))
size = (frame_width, frame_height)

mask_image = cv2.VideoWriter('Optical flow.avi',cv2.VideoWriter_fourcc('M','J','P','G'),20, size)
optical_flow_with_features = cv2.VideoWriter('Optical Flow with features.avi',cv2.VideoWriter_fourcc('M','J','P','G'),20, size)
features_video = cv2.VideoWriter('Features .avi',cv2.VideoWriter_fourcc('M','J','P','G'),20, size)


# Read  the first frame & convert the first frame to grayscale 
# Find the feature to use for optical flow using the find_features() function.
ret, old_frame = capture.read()
mask = np.zeros_like(old_frame)
prev_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
prev_points = find_features(prev_gray)

while(1):
    # Read the current frame
    ret, frame = capture.read()

    # Terminate while loop at the end of the video feed.
    if not ret:
        break

    # Call the Lucas Kannade model to find the optical flow of the features detected above using corner detector. 
    current_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    current_points, status, error = LK_model(prev_gray, current_gray, prev_points)

    # If the features are detected.
    if current_points is not None:
        current_points = current_points[status == 1]
        old_points = prev_points[status == 1]

    # Enumerate over all the feature points detected and draw a circle over the feature points detected
    # Also draw a line on the mask from the coordinates of the feature in the previous position to the current position
    for i, (current, prev) in enumerate(zip(current_points, old_points)):
        a, b = current.ravel()
        c, d = prev.ravel()
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
        frame = cv2.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)

    # Write the image to the respective cv2.VideoWriter() objects
    mask_image.write(mask)
    features_video.write(frame)
    img = cv2.add(frame, mask)
    optical_flow_with_features.write(img)


    k = cv2.waitKey(5) & 0xff
    if k == 27:
        break

    # Set the current frame to previous frame and current frame features to the previous frame features 
    prev_gray = current_gray.copy()
    prev_points = current_points.reshape(-1, 1, 2)

# Release all the cv2.VideoWriter() objects tand destroy all windows
capture.release()
mask_image.release()
optical_flow_with_features.release()
features_video.release()

cv2.destroyAllWindows()

